# Predicting phase behavior

This notebook will focus on using molecular dynamics (MD) simulation to predict the phase behavior of point particles that interact via the Lennard-Jones potential. 

To perform these simulations we will use HOOMD-Blue, as we can easily and quickly run this from the jupyter notebook.

## Single component Lennard-Jones system

First, let us consider a single component system composed of Lennard-Jones (LJ) spheres, identical to what we have previously examined in prior notebooks.  Here, the goal is to use simulation to perform simulations of the LJ particles in different phases: solid, liquid and gas. 

We will do this by modifying the density and temperature of the simulation to move to different points in the phase diagram. 

The follow paper provides a clear phase diagram that we can use to direct our simulations:
>T. E. Karakasidis, A. Fragkou, and A. Liakopoulos, "System dynamics revealed by recurrence quantification analysis: Application to molecular dynamics simulations" Phys. Rev. E 76, 021120 – Published 24 August 2007
https://journals.aps.org/pre/abstract/10.1103/PhysRevE.76.021120




In [ ]:
from IPython.display import Image
Image(url='img/lj_phase_diagram.png')  

A common way to help evaluate the phase of a system is to use the radial distribution function (RDF, sometimes referred to as g(r)). In the RDF, we create a histogram of the distance between all particle pairs, normalized by the density of the system multipied by the volume of the bin (i.e., $\rho *4 \pi r^2dr$ where $\rho$ is the number density and dr is the width of the bin). Systems where particle positions are strongly correlated will show larger values in the RDF. An example of the RDF for different phases is shown below.  See [this link](https://chem.libretexts.org/Bookshelves/Biological_Chemistry/Concepts_in_Biophysical_Chemistry_(Tokmakoff)/01%3A_Water_and_Aqueous_Solutions/01%3A_Fluids/1.02%3A_Radial_Distribution_Function) for more info on the RDF.

In [ ]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/commons/b/b8/Simulated_Radial_Distribution_Functions_for_Solid%2C_Liquid%2C_and_Gaseous_Argon.svg')

We'll need to modify the prior simulation script to allow us to change the box volume to be able to change density.  A tutorial for modifying the box that provides step-by-step details is available in the (HOOMD-Blue documentation)[https://hoomd-blue.readthedocs.io/en/v3.2.0/tutorial/01-Introducing-Molecular-Dynamics/03-Compressing-the-System.html]. 


The structure of the script is now slightly changed compared to the prior examples. In the simulation runtime section, we now have 3 distinct stages:
1) compression 
2) relaxation
3) production

The addition of the relaxation stage between compression and production is because we would like the system to reach a steady-state before we start collecting data for our final analysis. 

In [ ]:
import hoomd
import gsd.hoomd


cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)

#########################
# system initialization
#########################

sim.create_state_from_gsd(filename='datafiles/lj.gsd')

#########################
# interaction definition
#########################

cell = hoomd.md.nlist.Cell(buffer=1.0)
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('LJ', 'LJ')] = dict(epsilon=1, sigma=1)
lj.r_cut[('LJ', 'LJ')] = 2.5

#########################
# integrator setup
#########################

nvt = hoomd.md.methods.NVT(kT=1.0, filter=hoomd.filter.All(), tau=1.0)
integrator = hoomd.md.Integrator(dt=0.005)
integrator.forces.append(lj)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

#########################
# runtime parameters
#########################

# add a logger to output the current step and timesteps per second (TPS) 
logger = hoomd.logging.Logger(categories=['scalar', 'string'])
logger.add(sim, quantities=['timestep', 'tps'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)
sim.operations.writers.append(table)

# add a logger to output the thermodynamic data during the entire run 
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermodynamic_properties)

log_thermo = hoomd.logging.Logger()
log_thermo.add(thermodynamic_properties)
gsd_thermo = hoomd.write.GSD(filename='thermo.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             mode='wb',
                             filter=hoomd.filter.Null(),
                             log=log_thermo)
sim.operations.writers.append(gsd_thermo)

# ================
# compress the box
# ================

compression_time = 20000
final_rho = 1.2


ramp = hoomd.variant.Ramp(A=0, B=1, t_start=sim.timestep, t_ramp=compression_time)

initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)  # make a copy of initial_box
final_box.volume = sim.state.N_particles / final_rho
box_resize_trigger = hoomd.trigger.Periodic(10)
box_resize = hoomd.update.BoxResize(box1=initial_box,
                                    box2=final_box,
                                    variant=ramp,
                                    trigger=box_resize_trigger)
sim.operations.updaters.append(box_resize)
sim.run(compression_time+1)
if sim.state.box == final_box:
    print("system box size matches target")
    
sim.operations.updaters.remove(box_resize)
print("compression complete")

# ================
# relaxation
# ================

sim.run(50000)
print("relaxation complete")

# ================
# production 
# ================

log_traj = hoomd.logging.Logger()
gsd_traj = hoomd.write.GSD(filename='trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(5000),
                             mode='wb',
                             filter=hoomd.filter.All(),
                             log=log_traj)
sim.operations.writers.append(gsd_traj)

sim.run(50000)

hoomd.write.GSD.write(state=sim.state, filename='final.gsd', mode='wb')
print("simulation complete")

#delete the instances we defined ensure writers are closed
del sim, gsd_thermo, gsd_traj, thermodynamic_properties, log_thermo, log_traj, logger, table
del integrator, nvt, lj, cell, cpu

Note we output the timesteps per second (TPS) during the simulation run.  Higher values mean the code is running more efficiently.  For a system where we compress from a density of 0.125 (density of particles in the lj.gsd data file) to a higher density, the TPS value drops during the compression stage.  Recall the discussion of neighorlists and cutoffs; as the system density increases, the number of neighbors we need to consider will also increase. 

### Exercise 
Pick a few statepoints from the phase diagram to simulate a crystal phase, adjusting the density and temperature.   
 * Examine the thermodynamic output (see below) to ensure that the system has reached a steady state. 
 * Render the simulation trajectory as movie. Does the phase look like a well ordered solid?
 * Use the following code to calculate and plot the RDF. How does this compare the example plots above?
 * Repeat the same steps above, but choose a statepoint that would result in a fluid phase system.
 
 


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('default')
import mdtraj as md

In [ ]:
thermo_log = gsd.hoomd.open('thermo.gsd', 'rb')

time = []
pe = []
for frame in thermo_log:
    time.append(frame.configuration.step)
    pe.append(frame.log['md/compute/ThermodynamicQuantities/potential_energy'][0])

n = thermo_log[-1].log['md/compute/ThermodynamicQuantities/num_particles'][0]
vol = thermo_log[-1].log['md/compute/ThermodynamicQuantities/volume'][0]

print("density: ", n/vol)

In [ ]:
plt.plot(time, pe)
plt.ylabel('potential energy')
plt.xlabel('timestep')

plt.show()

In [ ]:
traj = md.load('trajectory.gsd')
traj.center_coordinates
r_traj, g_r_traj = md.compute_rdf(traj,  r_range=[0, traj.unitcell_lengths[0][1]/2.0], 
                                pairs = traj.top.select_pairs("all", "all"), 
                                  bin_width=0.01, periodic=True, opt=True)


In [ ]:
plt.plot(r_traj, g_r_traj)
plt.ylabel('g(r)')
plt.xlabel('r')

plt.show()

In [ ]:
from fresnel_render import render
from fresnel_render import render_movie

traj_log = gsd.hoomd.open('trajectory.gsd', 'rb')

render_movie(traj_log)